In [1]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, precision_score, f1_score, matthews_corrcoef
from collections import defaultdict
from scipy.spatial import distance
import pandas as pd
import numpy as np

In [2]:
# Cargar el conjunto de datos
credit = pd.read_csv('/Users/azulmakk/Universidad/Analisis Multivariado/Final/credit_risk_dataset.csv', header=0)

# Seleccionar columnas relevantes
selected_columns = ['loan_amnt', 'loan_percent_income', 'person_emp_length', 'loan_int_rate', 'loan_grade']
selected_credit = credit[selected_columns]

# Convertir columnas a tipo numérico
selected_credit.iloc[:, :4] = selected_credit.iloc[:, :4].apply(pd.to_numeric, errors='coerce')

# Eliminar filas con valores nulos
selected_credit = selected_credit.dropna()

# Convertir la columna 'loan_grade' a tipo de datos categórico
selected_credit['loan_grade'] = selected_credit['loan_grade'].astype('category')

# Seleccionar categorías específicas para el conjunto de muestra
selected_cat = ["A", "B", "C"]
sample = selected_credit[selected_credit['loan_grade'].isin(selected_cat)]

loan_grade_counts = sample['loan_grade'].value_counts()
print(f'Conteo de categorías de préstamos:\n{loan_grade_counts}\n')

# Dividir el conjunto de datos en características (X) y variable objetivo (y)
X = sample.iloc[:, :-1]
y = sample['loan_grade']

Conteo de categorías de préstamos:
loan_grade
A    9402
B    9151
C    5699
D       0
E       0
F       0
G       0
Name: count, dtype: int64



In [3]:
# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Construir el conjunto de redes neuronales mediante Bagging
num_networks = 15
num_hidden_units = 10
nnee_classifier = BaggingClassifier(base_estimator=MLPClassifier(hidden_layer_sizes=(num_hidden_units,)),
                                    n_estimators=num_networks, random_state=42)

# Entrenar el conjunto de redes neuronales
nnee_classifier.fit(X_train, y_train)

# Predecir las etiquetas de clase en el conjunto de prueba
y_pred = nnee_classifier.predict(X_test)

/Users/azulmakk/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [4]:
# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matriz de Confusión:")
print(conf_matrix)

# Calcular la exactitud del clasificador
accuracy = accuracy_score(y_test, y_pred)
print(f'Exactitud del clasificador NNEE: {accuracy:.2f}')

# Calcular la precisión
precision = precision_score(y_test, y_pred, average='weighted')
print(f'Precisión: {precision:.2f}')

# Calcular la puntuación F1
f1 = f1_score(y_test, y_pred, average='weighted')
print(f'F1 Score: {f1:.2f}')

# Calcular el coeficiente de correlación de Matthews (MCC)
mcc = matthews_corrcoef(y_test, y_pred)
print(f'Coeficiente de Correlación de Matthews: {mcc:.2f}')

Matriz de Confusión:
[[1302  544   26]
 [  92 1368  377]
 [   9  509  624]]
Exactitud del clasificador NNEE: 0.68
Precisión: 0.72
F1 Score: 0.69
Coeficiente de Correlación de Matthews: 0.52
